In [175]:
# Auto Reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [177]:
# Import Custom Modules
from webscrape import *
from helpers import *

In [4]:
# Create Dictionaries by Year with Empty DataFrame as Vaule
top_charts = {
    2015: pd.DataFrame(),
    2016: pd.DataFrame(),
    2017: pd.DataFrame(),
    2018: pd.DataFrame(),
    2019: pd.DataFrame(),
}

In [5]:
# 2019 Info
top_songs_2019 = get_top_songs(2019)
top_2019 = [get_full_lyrics(song, artist) for song, artist, rank in top_songs_2019]

In [6]:
# 2018 Info
top_songs_2018 = get_top_songs(2018)
top_2018 = [get_full_lyrics(song, artist) for song, artist, rank in top_songs_2018]

In [7]:
# 2017 Info
top_songs_2017 = get_top_songs(2017)
top_2017 = [get_full_lyrics(song, artist) for song, artist, rank in top_songs_2017]

In [8]:
# 2016 Info
top_songs_2016 = get_top_songs(2016)
top_2016 = [get_full_lyrics(song, artist) for song, artist, rank in top_songs_2016]

In [31]:
# 2015 Info
top_songs_2015 = get_top_songs(2015)
top_2015 = [get_full_lyrics(song, artist.partition('Featuring')[0]) for song, artist, rank in top_songs_2015]

In [78]:
# Year list
years = [2015, 2016, 2017, 2018, 2019]
# List of top song tuples by year
top_songs_list = [top_songs_2015, top_songs_2016, top_songs_2017, top_songs_2018, top_songs_2019]
# List of top song lyrics by year
top_lyrics_list = [top_2015, top_2016, top_2017, top_2018, top_2019]

In [80]:
# Create DataFrame
df = pd.DataFrame(
        columns=['song', 'artist', 'rank', 'year', 'lyrics'],
)

In [81]:
for year, info, lyrics in zip(years, top_songs_list, top_lyrics_list):
    x = pd.DataFrame(
        data=[[*info[x], year, lyrics[x]] for x in range(len(info))],
        columns=['song', 'artist', 'rank', 'year', 'lyrics'],)

    df = df.append(x, ignore_index=True)


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
song      250 non-null object
artist    250 non-null object
rank      250 non-null object
year      250 non-null object
lyrics    250 non-null object
dtypes: object(5)
memory usage: 4.9+ KB


In [184]:
title, artist = df['song'][247], df['artist'][247]

In [185]:
response = get_extract_response(title, artist)

In [194]:
# response

In [193]:
scrape_song_lyrics_from_url(response)

"(I want a slim, fine woman with some twerk with her)\nThrow that, twerk that\n(I want a slim, fine woman with some twerk with her)\nShake that, bounce that\n(I want a slim, fine woman with some twerk with her)\nCardi\nAyy, ayy, ayy, ayy, ayy, ayy, ayy\n(I want a slim, fine woman with some twerk with her)\n(Action)\n\nDo it, baby, stick it, baby\nMove it, baby, lick it, baby (Do it)\nSuck up on that clit until that pussy got a hickey, baby (Bounce)\nWatch big, coulda bought a Range Rover (Range Rover)\nChain little, but I spent some change on it (Change on it)\nNigga mad, I'ma put the gang on him (Gang on it)\nThey'll die 'bout me, they'll bang on him (Facts)\nAnd that ass poke out, lil frame on her (Ow)\nPussy so good he got my name on it (Miami)\nItty-bitty-pretty, I'm the realest in the city\nOnly fuckin' with the plug, got a nigga worth a billi'\nShowin' love, only talk about bands when he hit me\nChose him, he ain't pick me, and we never doin' quickies\n\n(I want a slim, fine woma